In [1]:
import game
import time
import copy
import player
import torch
import pygame
import random
import constants
import numpy as np
import pandas as pd
from numba import jit, njit
from player import RandomGuessAIPlayer

pygame 2.0.1 (SDL 2.0.14, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
df = pd.DataFrame(columns=['seed', 'max_val', 'score', 'runtime'])
df.set_index('seed', inplace=True)

In [3]:
%%time
for seed in range(100):
    print(f"seed: {seed}")
    g = game.Game(seed=seed)
    player = RandomGuessAIPlayer(
        game=g, searches_per_move=20, search_length=10)
    score, max_val, runtime = player.run()
    pygame.quit()
    df.loc[seed] = {"max_val": max_val, "score": score, "runtime": runtime}

seed: 0
Init Random guesser AI Player
done, 965 iterations, score: 16192, 34.29s, max value: 1024.0
seed: 1
Init Random guesser AI Player
done, 801 iterations, score: 14016, 27.69s, max value: 1024.0
seed: 2
Init Random guesser AI Player
done, 901 iterations, score: 15424, 31.2s, max value: 1024.0
seed: 3
Init Random guesser AI Player
done, 734 iterations, score: 12368, 25.13s, max value: 1024.0
seed: 4
Init Random guesser AI Player
done, 483 iterations, score: 7116, 16.73s, max value: 512.0
seed: 5
Init Random guesser AI Player
done, 480 iterations, score: 7060, 16.64s, max value: 512.0
seed: 6
Init Random guesser AI Player
done, 393 iterations, score: 5564, 13.75s, max value: 512.0
seed: 7
Init Random guesser AI Player
done, 474 iterations, score: 6872, 16.56s, max value: 512.0
seed: 8
Init Random guesser AI Player
done, 944 iterations, score: 16008, 32.72s, max value: 1024.0
seed: 9
Init Random guesser AI Player
done, 527 iterations, score: 7944, 18.31s, max value: 512.0
seed: 10
In

In [8]:
df[df["max_val"] == 2048]

,max_val,score,runtime
seed,,,
44,2048.0,20300,33.458039
65,2048.0,20532,34.455811
72,2048.0,20324,33.701174
81,2048.0,20224,33.584945


In [2]:
g = game.Game(seed=44)
player = RandomGuessAIPlayer(
    game=g, searches_per_move=20, search_length=10)
score, max_val, runtime = player.run(fps=100)

Init Random guesser AI Player
won, 961 iterations, score: 20300, 56.48s, max value: 2048.0


In [3]:
pygame.quit()